# Tensorflow 2 on CoCalc

This shows some examples from the [migration guide](https://www.tensorflow.org/alpha/guide/migration_guide).

To get TF2 in Cocalc:

* Select "New Project..."  → "Advanced ..." → "Custom Software" → scroll down to **"Tensorflow 2"**

In [3]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [4]:
import tensorflow_datasets as tfds

In [3]:
W = tf.Variable(tf.ones(shape=(2,2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")

@tf.function
def forward(x):
    return W * x + b

out_a = forward([1,0])
print(out_a)

tf.Tensor(
[[1. 0.]
 [1. 0.]], shape=(2, 2), dtype=float32)


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.02),
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

train_data = tf.ones(shape=(1, 28, 28, 1))
test_data = tf.ones(shape=(1, 28, 28, 1))

In [5]:
train_out = model(train_data, training=True)
print(train_out)

tf.Tensor([[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]], shape=(1, 10), dtype=float32)


In [7]:
datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [8]:
mnist_train.

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>